In [1]:
import os
import pandas as pd
import numpy as np

from pymongo import MongoClient
from pprint import pprint

ModuleNotFoundError: No module named 'pymongo'

In [150]:
pd.options.display.max_columns = 999
pd.options.display.max_rows = 999

# Writing Functions to clean the letters
## Opens file

In [2]:
def openFile(fn):
    
    file = open(fn,"r",encoding='utf-8-sig') 
    text = file.read()
    
    return text
    

## Gets dates from document

In [3]:
def getDates(text):
    
    dates = []
    letters = text.split('newletter')

    for letter in letters:
        letter = letter.split()
        for i in range(len(letter)):
            word = letter[i]
            if word in ['January', 'February', 'March', 'April', 'May', 'June', 'July', 'August', 'September',
                       'October', 'November', 'December']:
                    dates.append(letter[i:i+3])

    letter_dates = []

    for date in dates:
        if date[-1] in ['1942', '1943', '1944', '1945']:
            letter_dates.append(' '.join(date))

    return letter_dates
    

## Gets letters from document

In [4]:
def getContent(text):
    
    letters = text.lower().split('newletter')
    clean_letters = []

    for letter in letters:
        clean_letters.append(" ".join(letter.split()))
    
    return clean_letters

In [72]:
def getLocation(text):
    
    locations = ['australia', 'guinea', 'pacific', 'phillippine', 'mindanao', 'zamboanga', 'leyte', 
                 'okinawa', 'japan','hiro', 'california']
    
    letters = text.lower().split('newletter')
    
    letter_locations = []
    
    for letter in letters:
        for word in letter.split():
            if word in locations:
                letter_locations.append(word)
            break
    return letter_locations

# Listing all files in directory

In [5]:
for fn in os.listdir('letters_txt'):
    print(fn)

5-8_1944.txt
5-8_1945.txt
9-12_1942.txt
9-12_1943.txt
5-8_1943.txt
9-12_1944.txt
9-11_1945.txt
1-4_1943.txt
1-4_1944.txt
1-4_1945.txt
1_5-8_1942.txt


# Check to make sure number of dates matches number of letters

In [6]:
for fn in os.listdir('letters_txt'):
    
    test_file = openFile('letters_txt/' + fn)
    test_dates = getDates(test_file)
    test_content = getContent(test_file)
    
    print(fn, '-->', len(test_dates) == len(test_content))
    

5-8_1944.txt --> True
5-8_1945.txt --> True
9-12_1942.txt --> True
9-12_1943.txt --> True
5-8_1943.txt --> True
9-12_1944.txt --> True
9-11_1945.txt --> True
1-4_1943.txt --> True
1-4_1944.txt --> True
1-4_1945.txt --> True
1_5-8_1942.txt --> True


# Build documents

In [7]:
dataframe = []

for fn in os.listdir('letters_txt'):
    
    text = openFile('letters_txt/' + fn)
    dates = getDates(text)
    content = getContent(text)
    
    for date, letter in zip(dates, content):
    
        dictionary = {}
        dictionary['Date'] = date
        dictionary['Letter'] = letter

        dataframe.append(dictionary)


## Building MongoDB database

In [20]:
client = MongoClient()

In [21]:
mydb = client["mydatabase"]

In [22]:
mycollection = mydb["WW2_letters"]

In [23]:
mycollection.insert_many(dataframe)

In [24]:
cursor = mycollection.find({},{ "_id": 0, "Date": 1, "Letter": 1 })

In [25]:
letters_df = pd.DataFrame(list(cursor))

In [37]:
letters_df.Date =  pd.to_datetime(letters_df.Date)

In [43]:
letters_df = letters_df.sort_values("Date").reset_index()

In [45]:
letters_df.drop('index', axis=1, inplace=True)

In [46]:
letters_df.head()

,Date,Letter
0,1942-05-13,"australia may 13, 1942 my darling annette, tod..."
1,1942-05-19,"western union telegram australia may 19, 1942 ..."
2,1942-05-20,"may 20, 1942 my darling annette, to my great s..."
3,1942-05-22,"silvio may 22, 1942 my darling annette, it’s 8..."
4,1942-06-12,"australia june 12, 1942 my darling annette, i ..."


## Getting locations

In [159]:
locations = ['australia', 'guinea', 'pacific', 'philippine', 'mindanao', 'zamboanga', 'leyte', 
                 'okinawa', 'japan','hiro', 'california']

letter_locals = []

for index, letter in enumerate(letters_df.Letter):

    for word in letter.split():
        if word not in locations:
            continue
        else:
            letter_locals.append(word)
            break
            
    if word not in locations:
        letter_locals.append(letter_locals[index - 1])


In [163]:
letters_df.Location = np.array(letter_locals)

In [166]:
locations_dict = {'australia': 'Australia', 'guinea': 'New Guinea', 'pacific': 'Southwest Pacific', 
                  'philippine': 'Philippine Islands', 'mindanao': 'Mindanao', 'zamboanga':'Zamboanga' , 
                  'leyte': 'Leyte', 'okinawa':'Okinawa' , 'japan': 'Japan', 'hiro': 'Hiro', 
                  'california': 'California'}

In [167]:
letters_df.Location = letters_df.Location.apply(lambda x: locations_dict[x])

In [174]:
letters_df = letters_df[['Date', 'Location', 'Letter']]

In [176]:
letters_df.head()

,Date,Location,Letter
0,1942-05-13,Australia,"australia may 13, 1942 my darling annette, tod..."
1,1942-05-19,Australia,"western union telegram australia may 19, 1942 ..."
2,1942-05-20,Australia,"may 20, 1942 my darling annette, to my great s..."
3,1942-05-22,Australia,"silvio may 22, 1942 my darling annette, it’s 8..."
4,1942-06-12,Australia,"australia june 12, 1942 my darling annette, i ..."


In [178]:
#letters_df.to_csv('letters_df.csv')